In [1]:
# imports
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [3]:
# calibrate camera
objp = np.zeros((6*9,3), np.float32)
objp[:, :2] = np.mgrid[:9, :6].T.reshape(-1, 2)

objpoints = []
imgpoints = []

images = glob.glob('camera_cal/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)
    
    ret, mtx, dist, _, _ = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    if ret:
        p = {}
        p['cmtx'] = mtx
        p['dCoefs'] = dist
        pickle.dump(p, open('camera_cal/camera_params.p', 'wb'))
        